In [15]:
import numpy as np
from sklearn.metrics import roc_curve

from collections import defaultdict

#dataset_configs = {
    #"academicodec_hifi_16k_320d": {"only_include_id_patterns":["p227","p228"]},
    #"audiodec_24k_320d": {"only_include_id_patterns":["p227","p228"]},
    #"descript-audio-codec-16khz": {"only_include_id_patterns":["p227","p228"]},
    #"encodec_24khz": {"only_include_id_patterns":["p227","p228"]},
    #"funcodec-funcodec_en_libritts-16k-nq32ds320": {"only_include_id_patterns":["p227","p228"]},
    #"llamapartialspoof_r01tts0a": {},
    #"vctk": {}
#}

dataset_configs = {
#    "academicodec_hifi_16k_320d": {"only_include_id_patterns":["p227","p228"]},
#    "audiodec_24k_320d": {"only_include_id_patterns":["p227","p228"]},
#    "descript-audio-codec-16khz": {"only_include_id_patterns":["p227","p228"]},
#    "encodec_24khz": {"only_include_id_patterns":["p227","p228"]},
#    "funcodec-funcodec_en_libritts-16k-nq32ds320": {"only_include_id_patterns":["p227","p228"]},
    "asvspoof2021_df": {},
    "ami_ihm": {},
    "ami_sdm": {},
    "asvspoof2019_la": {},
#    "av_deepfake_1m": {},
    "emofake": {},
#    "fakeavceleb": {},
    "librispeech_test_clean": {},
    "librispeech_test_other": {},
    "llamapartialspoof_r01tts0a": {},
    "llamapartialspoof_r01tts0b": {},
#    "mlaad": {},
#    "partialspoof": {},
#    "release_in_the_wild": {},
#    "scenefake": {},
#    "speech_tokenizer": {},
    "vctk": {},
}
model_name = "conformer-based-classifier-for-anti-spoofing"

id2meta = defaultdict(lambda: defaultdict( lambda: defaultdict(dict) )) # 3-levels nested default dict
id2count = defaultdict(lambda: defaultdict( lambda: defaultdict(dict) )) # 3-levels nested default dict
for dataset_name, config in dataset_configs.items():
    
    with open(f"../eer_files_600/{model_name}/scores/{dataset_name}.txt", 'r') as rf:
        lines = rf.readlines()
        for i, line in enumerate(lines):
            if line == "\n": continue
            split = line.strip().split(' ')
            id_ = split[0]
            score = float(split[1])
            
            #if "DF_E_2101080" in line: continue
            if config.get("only_include_id_patterns", None) is not None:
                if all([_ not in id_ for _ in config["only_include_id_patterns"]]):
                    continue

            id2meta[model_name][dataset_name][id_]["score"] = score

            if " eval " in line or " eval\n" in line: assert split != "eval"
    
    with open(f"../eer_files_600/{model_name}/keys/{dataset_name}.txt", 'r') as rf:
        lines = rf.readlines()
        for i, line in enumerate(lines):
            if line == "\n": continue
            split = line.strip().split(' ')
            id_ = split[1]
            data_type = split[7]
            label = split[5]
            codec = split[2]
            synthesizer = split[4]

            if config.get("only_include_id_patterns", None) is not None:
                if all([_ not in id_ for _ in config["only_include_id_patterns"]]):
                    continue
            
            if data_type == "eval":
                id2meta[model_name][dataset_name][id_]["label"] = label
                id2meta[model_name][dataset_name][id_]["codec"] = codec
                if synthesizer != "-":
                    id2meta[model_name][dataset_name][id_]["synthesizer"] = synthesizer
                    id2count[model_name][dataset_name]["synthesizer"][synthesizer] = \
                        id2count[model_name][dataset_name]["synthesizer"].get(synthesizer, 0) + 1
                if codec != "-" and codec != "nocodec":
                    id2count[model_name][dataset_name]["codec"][codec] = \
                        id2count[model_name][dataset_name]["codec"].get(codec, 0) + 1
                if label == "bonafide" and "score" in id2meta[model_name][dataset_name][id_]:
                    id2count[model_name][dataset_name]["bonafide"] = id2count[model_name][dataset_name].get("bonafide", 0) + 1
                elif label == "spoof" and "score" in id2meta[model_name][dataset_name][id_]:
                    id2count[model_name][dataset_name]["spoof"] = id2count[model_name][dataset_name].get("spoof", 0) + 1

            if " eval " in line or " eval\n" in line: assert split != "eval"
            
    print(f"Finish reading meta info for {dataset_name}")

Finish reading meta info for asvspoof2021_df
Finish reading meta info for ami_ihm
Finish reading meta info for ami_sdm
Finish reading meta info for asvspoof2019_la
Finish reading meta info for emofake
Finish reading meta info for librispeech_test_clean
Finish reading meta info for librispeech_test_other
Finish reading meta info for llamapartialspoof_r01tts0a
Finish reading meta info for llamapartialspoof_r01tts0b
Finish reading meta info for vctk


In [11]:
id2meta["conformer-based-classifier-for-anti-spoofing"]["llamapartialspoof_r01tts0a"]["dev-clean_1462_170145_000020_000002"]

{}

In [12]:
id2count["conformer-based-classifier-for-anti-spoofing"]["academicodec_hifi_16k_320d"]["spoof"]

{}

In [16]:
def get_subset(bonafide_patterns=[None,None,None], spoof_patterns=[None,None,None], synthesizer_patterns=[None]):
    y, score = [], []
    counts = [0,0] # spoof count, bona fide count
    for model in id2meta.keys():
        if spoof_patterns[0] is None or spoof_patterns[0] in model:
            for dataset in id2meta[model].keys():
                if spoof_patterns[1] is None or spoof_patterns[1] in dataset:
                    for id_ in id2meta[model][dataset].keys():
                        if spoof_patterns[2] is None or spoof_patterns[2] in id_:
                            if id2meta[model][dataset][id_].get("label", None) == "spoof":
                                if synthesizer_patterns == [None] or any([(_ in id2meta[model][dataset][id_].get("synthesizer", [])) for _ in synthesizer_patterns]):
                                    try:
                                        score.append(id2meta[model][dataset][id_]["score"])
                                        y.append(0)
                                        counts[0] += 1
                                    except Exception as e:
                                        print(id_)
                                        print(e)
        if bonafide_patterns[0] is None or bonafide_patterns[0] in model:
            for dataset in id2meta[model].keys():
                if bonafide_patterns[1] is None or bonafide_patterns[1] in dataset:
                    for id_ in id2meta[model][dataset].keys():
                        if bonafide_patterns[2] is None or bonafide_patterns[2] in dataset:
                            if id2meta[model][dataset][id_].get("label", None) == "bonafide":
                                try:
                                    score.append(id2meta[model][dataset][id_]["score"])
                                    y.append(1)
                                    counts[1] += 1
                                except Exception as e:
                                    print(id_)
                                    print(e)
    print(counts)
    return y, score

def get_eer(y, score):
    fpr, tpr, threshold = roc_curve(y, score, pos_label=1)
    fnr = 1 - tpr
    eer_threshold = threshold[np.nanargmin(np.absolute((fnr - fpr)))]
    EER = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
    eer_precision = fnr[np.nanargmin(np.absolute((fnr - fpr)))] - fpr[np.nanargmin(np.absolute((fnr - fpr)))]
    assert eer_precision < 0.02, eer_precision
    print("eer threshold =", eer_threshold)
    return EER

In [17]:
get_eer(*get_subset())

[79640, 4607]
eer threshold = -3.570448875427246


0.05183324962330487

In [18]:
eers = []
bonanide_subsets = []
for dataset in id2count["conformer-based-classifier-for-anti-spoofing"].keys():
    bonafide_count = id2count["conformer-based-classifier-for-anti-spoofing"][dataset]["bonafide"]
    if type(bonafide_count) is int and bonafide_count > 0:
        bonanide_subsets.append(dataset)
print(bonanide_subsets)
spoof_subsets = []
for dataset in id2count["conformer-based-classifier-for-anti-spoofing"].keys():
    spoof_count = id2count["conformer-based-classifier-for-anti-spoofing"][dataset]["spoof"]
    if type(spoof_count) is int and spoof_count > 0:
        spoof_subsets.append(dataset)
print(spoof_subsets)

for bonafide_subset in bonanide_subsets:
    eers_ = []
    for spoof_subset in spoof_subsets:
        synthesizers = id2count["conformer-based-classifier-for-anti-spoofing"][spoof_subset]["synthesizer"].keys()
        if len(synthesizers) == 0:
            synthesizers = [None]
        for synthesizer in synthesizers:
            print(bonafide_subset, spoof_subset, synthesizer)
            eers_.append(get_eer(*get_subset([None,bonafide_subset,None], [None,spoof_subset,None], synthesizer_patterns=[synthesizer])))
    eers.append(eers_)

print(eers)

['asvspoof2021_df', 'ami_ihm', 'ami_sdm', 'asvspoof2019_la', 'emofake', 'librispeech_test_clean', 'librispeech_test_other', 'vctk']
['asvspoof2021_df', 'asvspoof2019_la', 'emofake', 'llamapartialspoof_r01tts0a', 'llamapartialspoof_r01tts0b']
asvspoof2021_df asvspoof2021_df HUB-D04
[600, 407]
eer threshold = -3.8930344581604004
asvspoof2021_df asvspoof2021_df HUB-N11
[600, 407]
eer threshold = -3.8855721950531006
asvspoof2021_df asvspoof2021_df Task1-team20
[600, 407]
eer threshold = -3.733920097351074
asvspoof2021_df asvspoof2021_df Task2-team09
[600, 407]
eer threshold = -3.8987796306610107
asvspoof2021_df asvspoof2021_df Task2-team12
[600, 407]
eer threshold = -3.8995535373687744
asvspoof2021_df asvspoof2021_df HUB-N16
[600, 407]
eer threshold = -3.897233724594116
asvspoof2021_df asvspoof2021_df Task1-team01
[600, 407]
eer threshold = -3.972980499267578
asvspoof2021_df asvspoof2021_df Task1-team27
[600, 407]
eer threshold = -3.635981559753418
asvspoof2021_df asvspoof2021_df Task2-tea

In [19]:
data = np.array(eers, dtype='float').T

In [29]:
data = np.array(eers, dtype='float')
print(np.amax(data, axis=1))
print(np.mean(np.amax(data, axis=1)))
max_eer_row = np.concatenate([np.amax(data, axis=1), np.mean(np.amax(data, axis=1), keepdims=True)], axis=0)
print(max_eer_row)

[0.33333333 0.48833333 0.43       0.04833333 0.39666667 0.35666667
 0.39166667 0.085     ]
0.31625000000000003
[0.33333333 0.48833333 0.43       0.04833333 0.39666667 0.35666667
 0.39166667 0.085      0.31625   ]


In [33]:
data = np.array(eers, dtype='float')
print(np.mean(data, axis=1))
print(np.mean(np.mean(data, axis=1)))
avg_eer_row = np.concatenate([np.mean(data, axis=1), np.mean(np.mean(data, axis=1), keepdims=True)], axis=0)
print(avg_eer_row)

[0.03240656 0.07441932 0.08001725 0.0005588  0.04466273 0.02232329
 0.05507688 0.0022108 ]
0.03895945331317438
[0.03240656 0.07441932 0.08001725 0.0005588  0.04466273 0.02232329
 0.05507688 0.0022108  0.03895945]


In [23]:
from __future__ import absolute_import
from __future__ import print_function

import os
import sys
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.pyplot import cm


__author__ = "Xin Wang"
__email__ = "wangxin@nii.ac.jp"
__copyright__ = "Copyright 2020, Xin Wang"

#####################
## Latex table
## Derived from https://github.com/nii-yamagishilab/project-NN-Pytorch-scripts/blob/master/tutorials/plot_tools/table_API.py
#####################
def return_one_row_latex(content_buffer):
    return " & ".join(content_buffer) + r"\\ " + "\n"
        
def return_one_row_text(content_buffer):
    return " ".join(content_buffer) + "\n"

def fill_cell(text, length, sep=''):
    return "{str:^{wid}}".format(str=text, wid=length) + sep
    
def wrap_value(data, wrap_factor=0):
    if wrap_factor == 0:
        return data
    else:
        ratio = (1+wrap_factor) / (1-wrap_factor)
        return np.power((1 - np.power(1 - data, ratio)), 1/ratio)

def return_latex_color_cell(value, val_min, val_max, scale, wrap, color_func):
    
    # clip the value for color rendering
    value = np.clip(value, val_min, val_max)
    
    # normalized value
    if scale < 0:
        value = wrap_value((value - val_min) / (val_max - val_min), wrap)*-scale
        value = -scale - value
    else:
        value = wrap_value((value - val_min) / (val_max - val_min), wrap)*scale

    # only use RGB, not RGBA
    color_code = color_func(value)[:-1]
    
    color_code = ', '.join(["{:0.2f}".format(x) for x in color_code])
    return r"\cellcolor[rgb]{" + color_code + "}"

def is_valid_float(val):
    try:
        float(val)
    except ValueError:
        return False
    else:
        if val != np.inf and val == val:
            return True
        else:
            return False

def return_valid_number_idx(data_array):
    """return the index of data ceil that has valid nummerical value
    """
    is_numeric_3 = np.vectorize(is_valid_float, otypes = [bool])
    return is_numeric_3(data_array)

    
def print_table(data_array, column_tag, row_tag, 
                print_format = "1.2f", 
                with_color_cell = True,
                colormap='Greys', 
                colorscale = 0.5, 
                colorwrap = 0, 
                col_sep = '', 
                print_latex_table=True, 
                print_text_table=True,
                print_format_along_row=True,
                color_minmax_in = 'global',
                pad_data_column = 0,
                pad_dummy_col = 0):
    """
    print a latex table given the data (np.array) and tags    
    step1. table will be normalized so that values will be (0, 1.0)
    step2. each normalzied_table[i,j] will be assigned a RGB color tuple 
           based on color_func( normalzied_table[i,j] * color_scale)
    input
    -----
      data_array: np.array [M, N]
      column_tag: list of str, length N, tag in the first row
      row_tag: list of str, length M, tags in first col of each row
      
      print_format: str or list of str, specify the format to print number
                    default "1.2f"
      print_format_along_row: bool, when print_format is a list, is this
                    list specified for rows? Default True
                    If True, row[n] will use print_format[n]
                    If False, col[n] will use print_format[n]
      with_color_cell: bool, default True,
                      whether to use color in each latex cell
      colormap: str, color map name (matplotlib)
      colorscale: float, default 0.5, 
                    normalized table value will be scaled 
                    color = color_func(nomrlized_table[i,j] * colorscale)
                  list of float
                    depends on configuration of color_minmax_in
                    if color_minmax_in = 'row', colorscale[i] for the i-th row
                    if color_minmax_in = 'col', colorscale[j] for the j-th row
                  np.array
                    color_minmax_in cannot be 'row' or 'col'. 
                    colorscale[i, j] is used for normalized_table[i, j]
      colorwrap: float, default 0, wrap the color-value mapping curve
                 colorwrap > 0 works like mels-scale curve
      col_sep: str, additional string to separate columns. 
               You may use '\t' or ',' for CSV
      print_latex_table: bool, print the table as latex command (default True)
      print_text_table: bool, print the table as text format (default True)
      color_minmax_in: how to decide the max and min to compute cell color?
                 'global': get the max and min values from the input matrix 
                 'row': get the max and min values from the current row
                 'col': get the max and min values from the current column
                  (min, max): given the min and max values
                 default is global
      pad_data_column: int, pad columns on the left or right of data matrix
                  (the tag column will still be on the left)
                  0: no padding (default)
                  -N: pad N dummy data columns to the left
                   N: pad N dummy data columns to the right
      pad_dummy_col: int, pad columns to the left or right of the table
                  (the column will be padded to the left of head column)
                  0: no padding (default)
                  N: pad N columns to the left
    output
    ------
      latext_table, text_table
      
    Tables will be printed to the screen.
    The latex table will be surrounded by begin{tabular}...end{tabular}
    It can be directly pasted to latex file.
    However, it requires usepackage{colortbl} to show color in table cell.    
    """
    if column_tag is None:
        column_tag = ["" for data in data_array[0, :]]
    if row_tag is None:
        row_tag = ["" for data in data_array]

    if pad_data_column < 0:
        column_tag = ["" for x in range(-pad_data_column)] + column_tag
        dummy_col = np.zeros([data_array.shape[0], -pad_data_column]) + np.nan
        data_array = np.concatenate([dummy_col, data_array], axis=1)
    elif pad_data_column > 0:
        column_tag = ["" for x in range(pad_data_column)] + column_tag
        dummy_col = np.zeros([data_array.shape[0], pad_data_column]) + np.nan
        data_array = np.concatenate([data_array, dummy_col], axis=1)
    else:
        pass

    # check print_format
    if type(print_format) is not list:
        if print_format_along_row:
            # repeat the tag
            print_format = [print_format for x in row_tag]
        else:
            print_format = [print_format for x in column_tag]
    else:
        if print_format_along_row:
            assert len(print_format) == len(row_tag)
        else:
            assert len(print_format) == len(column_tag)


    # color configuration
    color_func = cm.get_cmap(colormap)
    #data_idx = return_valid_number_idx(data_array)    
    #value_min = np.min(data_array[data_idx])
    #value_max = np.max(data_array[data_idx])
    
    def get_latex_color(data_array, row_idx, col_idx, color_minmax_in):
        x = data_array[row_idx, col_idx]
        if color_minmax_in == 'row':
            data_idx = return_valid_number_idx(data_array[row_idx])
            value_min = np.min(data_array[row_idx][data_idx])
            value_max = np.max(data_array[row_idx][data_idx])
            if type(colorscale) is list:
                colorscale_tmp = colorscale[row_idx]
        elif color_minmax_in == 'col':
            data_idx = return_valid_number_idx(data_array[:, col_idx])
            value_min = np.min(data_array[:, col_idx][data_idx])
            value_max = np.max(data_array[:, col_idx][data_idx])    
            if type(colorscale) is list:
                colorscale_tmp = colorscale[col_idx]
        elif type(color_minmax_in) is tuple or type(color_minmax_in) is list:
            value_min = color_minmax_in[0]
            value_max = color_minmax_in[1]
            if type(colorscale) is np.ndarray:
                colorscale_tmp = colorscale[row_idx, col_idx]
        else:
            data_idx = return_valid_number_idx(data_array)
            value_min = np.min(data_array[data_idx])
            value_max = np.max(data_array[data_idx])
            if type(colorscale) is np.ndarray:
                colorscale_tmp = colorscale[row_idx, col_idx]
            
        if type(colorscale) is not list:
            colorscale_tmp = colorscale
            

        # return a color command for latex cell
        return return_latex_color_cell(x, value_min, value_max, 
                                       colorscale_tmp, colorwrap, color_func)
    
    # maximum width for tags in 1st column
    row_tag_max_len = max([len(x) for x in row_tag])

    # maximum width for data and tags for other columns
    if print_format_along_row:
        tmp_len = []
        for idx, data_row in enumerate(data_array):
            tmp_len.append(
                max([len("{num:{form}}".format(num=x, form=print_format[idx])) \
                     for x in data_row]))
    else:
        tmp_len = []
        for idx, data_col in enumerate(data_array.T):
            tmp_len.append(
                max([len("{num:{form}}".format(num=x, form=print_format[idx])) \
                     for x in data_col]))
    col_tag_max_len = max([len(x) for x in column_tag] + tmp_len)
    
    # prepare buffer
    text_buffer = ""
    latex_buffer = ""
    text_cell_buffer = []
    latex_cell_buffer = []

    # latex head
    if pad_dummy_col > 0:
        latex_buffer += r"\begin{tabular}{" \
                        + ''.join(['c' for x in column_tag + ['']])
        latex_buffer += ''.join(['c' for x in range(pad_dummy_col)]) + r"}"+"\n"
    else:
        latex_buffer += r"\begin{tabular}{" \
                        + ''.join(['c' for x in column_tag + ['']]) + r"}"+"\n"

    latex_buffer += r"\toprule" + "\n"
    
    # head row
    #  for latex
    hrow = [fill_cell("", row_tag_max_len)] \
           + [fill_cell(x, col_tag_max_len) for x in column_tag]
    if pad_dummy_col > 0:
        hrow = [fill_cell("", 1) for x in range(pad_dummy_col)] + hrow

    latex_buffer += return_one_row_latex(hrow)
    latex_buffer += r"\midrule" + "\n"

    latex_cell_buffer.append(hrow)

    #  for plain text (add additional separator for each column)
    hrow = [fill_cell("", row_tag_max_len, col_sep)] \
           + [fill_cell(x, col_tag_max_len, col_sep) for x in column_tag]
    text_buffer += return_one_row_text(hrow)
    text_cell_buffer.append(hrow)

    # contents
    row = data_array.shape[0]
    col = data_array.shape[1]
    for row_idx in np.arange(row):
        # row head
        row_content_latex = [fill_cell(row_tag[row_idx], row_tag_max_len)]
        row_content_text = [fill_cell(row_tag[row_idx],row_tag_max_len,col_sep)]
        
        if pad_dummy_col > 0:
            row_content_latex = [fill_cell("", 1) for x in range(pad_dummy_col)] \
                                + row_content_latex

        # each column in the raw
        for col_idx in np.arange(col):

            if print_format_along_row:
                tmp_print_format = print_format[row_idx]
            else:
                tmp_print_format = print_format[col_idx]

            if is_valid_float(data_array[row_idx,col_idx]):
                num_str = "{num:{form}}".format(num=data_array[row_idx,col_idx],
                                                form=tmp_print_format)
                latex_color_cell = get_latex_color(data_array, row_idx, col_idx,
                                                   color_minmax_in)
            elif type(data_array[row_idx,col_idx]) is str:
                num_str = "{num:{form}}".format(num=data_array[row_idx,col_idx],
                                                form=tmp_print_format)
                latex_color_cell = ''
            else:
                num_str = ''
                latex_color_cell = ''
                
            if not with_color_cell:
                latex_color_cell = ''
                
            row_content_text.append(
                fill_cell(num_str, col_tag_max_len, col_sep))

            row_content_latex.append(
                fill_cell(latex_color_cell + ' ' + num_str, col_tag_max_len))
            
        # latex table content
        latex_buffer += return_one_row_latex(row_content_latex)
        latex_cell_buffer.append(row_content_latex)
        # text content
        text_buffer += return_one_row_text(row_content_text)
        text_cell_buffer.append(row_content_text)

    latex_buffer += r"\bottomrule" + "\n"
    latex_buffer += r"\end{tabular}" + "\n"

    if print_latex_table:
        print(latex_buffer)
    if print_text_table:
        print(text_buffer)
    return latex_buffer, text_buffer, latex_cell_buffer, text_cell_buffer

In [25]:
print_table(data, ['x']*data.shape[1], ['x']*data.shape[0])

\begin{tabular}{cccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc}
\toprule
  &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x 

/tmp/ipykernel_813/2286276119.py:178: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  color_func = cm.get_cmap(colormap)


('\\begin{tabular}{cccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc}\n\\toprule\n  &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x   &  x  

In [76]:
import seaborn
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

seaborn.set(font_scale=0.7)

data = np.array(eers, dtype='float').T
print(data.shape)
normalized = data #/ data.sum()

spoof_subsets = [f"s{_}" for _ in range(1,data.shape[0]+1)]
bonafide_subsets = [f"b{_}" for _ in range(1,data.shape[1]+1)]

plt.figure(dpi=1200, figsize=(10, 10))
ax = seaborn.heatmap(
    normalized, robust=True, #vmin=0, vmax=0.3,
    xticklabels=bonafide_subsets, 
    square=True, cmap='Blues',
    cbar_kws={'format': FuncFormatter(lambda x, _: "%.0f%%" % (x * 100))}
)
ax.set_aspect(8/158)
#annot=data,
#yticklabels=spoof_subsets,

ax.set_xlabel('Bona fide Subset')
ax.set_ylabel('Spoof Subset')
ax.xaxis.tick_top()
ax.xaxis.set_label_position('top')
plt.tick_params(top=False, bottom=False, left=False, right=False)
plt.yticks(rotation=0)
plt.show()

(6, 1)
